# Main Library

In [1]:
# Reading Data & Visualization & Encoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score

# Pytorch
from  torch import Tensor
from torch.nn import Linear, ReLU, Sigmoid, Softmax, Module, BCELoss, CrossEntropyLoss
from torch.optim import SGD, Adam, NAdam
from torch.utils.data import DataLoader, Dataset, random_split

In [2]:
class Fetch_Data():
    def __init__(self, path):
        data = pd.read_csv(path)
        self.x = data.iloc[:, :-1].values
        self.x = self.x.astype("float32")
        
        self.y = data.iloc[:, -1].values
        self.y = OneHotEncoder().fit_transform(self.y.reshape(-1, 1)).toarray()
        self.y = self.y.astype("float32")
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def split(self, ratio=0.5):
        size1 = round(ratio * len(self.x))
        size2 = len(self.x) - size1
        
        return random_split(self, [size1, size2])

In [3]:
def Get_data(path, ratio):
    model = Fetch_Data(path)
    
    train, test = model.split(ratio=ratio)
    
    train = DataLoader(train, shuffle=True, batch_size=50)
    
    test = DataLoader(test, shuffle=False, batch_size=1000)
    
    return train, test

In [4]:
class NeuralNetwork(Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        
        self.layer1 = Linear(input_size, 128)
        self.activation1 = ReLU()
        
        self.layer2 = Linear(128, 32)
        self.activation2 = ReLU()
        
        self.layer3 = Linear(32, 8)
        self.activation3 = ReLU()
        
        self.layer4 = Linear(8, output_size)
        self.activation4 = Softmax(dim=-1)
        
        
    def forward(self, data):
        data = self.layer1(data)
        data = self.activation1(data)
        
        data = self.layer2(data)
        data = self.activation2(data)
        
        data = self.layer3(data)
        data = self.activation3(data)
        
        data = self.layer4(data)
        data = self.activation4(data)
        
        return data
        
        
def trainingData(train_data, model):
    loss_func = CrossEntropyLoss()
    optamizer = Adam(model.parameters(), lr=0.01)
    for epochs in range(100):
        for  (x, y) in train_data:
            optamizer.zero_grad()

            result = model(x)
            loss = loss_func(result, y)

            loss.backward()
            optamizer.step()
            
            
def Evaluation(test_data, model):
    accumelate_result = []
    accumelate_y = []
    for (x, y) in test_data:
        result = model(x)
        result = result.detach().numpy()
        y = y.detach().numpy()
        result = result.round()
        
        accumelate_result.append(result)
        accumelate_y.append(y)
        
    accumelate_result = np.vstack(accumelate_result)
    accumelate_y = np.vstack(accumelate_y)    

    print("The Accuracy Score is ", round(accuracy_score(accumelate_y, accumelate_result), 2) * 100, "%")

    
    
def get_class(value):
    if value == 0:
        return "Iris-setosa"
    elif value == 1:
        return "Iris-versicolor"
    else:
        return "Iris-virginica"
    
    
def prediction(attr, model):
    row = Tensor(attr)
    result = model(row)
    
    return get_class(np.argmax(result.detach().numpy()))

In [5]:
path = r"D:\Courses language programming\5_Machine Learning\Dataset For Machine Learning\Training_Data_set_For_ML\Iris.csv"

train, test = Get_data(path=path, ratio=0.8)

input_size = len(train.dataset[0][0])
output_size = len(train.dataset[0][1])
model = NeuralNetwork(input_size, output_size)

trainingData(train, model)

Evaluation(test, model)

The Accuracy Score is  97.0 %


In [6]:
prediction([6.7,3.1,5.6,2.4], model)

'Iris-virginica'